In [24]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

import wrangle as w
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [25]:
def insurance(data):
    
    origin_df = pd.read_csv(data)

    df = origin_df.copy()
    
    numerical = df.select_dtypes(include='number').columns.tolist()
    categorical = df.select_dtypes(exclude='number').columns.tolist()
    to_encode = ('sex', 'smoker')
    
    categorical = [col for col in categorical if col not in to_encode]

    for cat in to_encode:
        df[f'{cat}_encoded'] = df[cat].apply(lambda x: 0 if x in ['no', 'female'] else 1)
    
    dummies_df = pd.get_dummies(df['region'], drop_first=False, dtype='int')

    df = pd.concat([df,dummies_df], axis = 1)


    return origin_df, df

In [26]:
origin_df, df= insurance('insurance.csv')


In [27]:
df

,age,sex,bmi,children,smoker,region,charges,sex_encoded,smoker_encoded,northeast,northwest,southeast,southwest
0,19,female,27.900,0,yes,southwest,16884.92400,0,1,0,0,0,1
1,18,male,33.770,1,no,southeast,1725.55230,1,0,0,0,1,0
2,28,male,33.000,3,no,southeast,4449.46200,1,0,0,0,1,0
3,33,male,22.705,0,no,northwest,21984.47061,1,0,0,1,0,0
4,32,male,28.880,0,no,northwest,3866.85520,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830,1,0,0,1,0,0
1334,18,female,31.920,0,no,northeast,2205.98080,0,0,1,0,0,0
1335,18,female,36.850,0,no,southeast,1629.83350,0,0,0,0,1,0
1336,21,female,25.800,0,no,southwest,2007.94500,0,0,0,0,0,1


In [28]:
origin_df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [29]:
train, validate, test = w.data_split(df)
train.shape, validate.shape, test.shape

((936, 13), (201, 13), (201, 13))

In [30]:
def model_split(train, validate, test):

    X_train= train.drop(columns = ['sex','smoker','region','charges'])
    y_train = train.charges
    
    
    X_validate = validate.drop(columns = ['sex','smoker','region','charges'])
    y_validate = validate.charges
    
    X_test = test.drop(columns = ['sex','smoker','region','charges'])
    y_test = test.charges



    return X_train, y_train, X_validate, y_validate, X_test, y_test


In [31]:
X_train, y_train, X_validate, y_validate, X_test, y_test = model_split (train,validate, test)

In [32]:
def scaled_data(X_train, X_validate, X_test):
    
    X_train_scaled = X_train.copy()
    X_validate_scaled = X_validate.copy()
    X_test_scaled = X_test.copy()    
    
    scaler = StandardScaler()
    
    X_train_scaled[['age','bmi','children']] = scaler.fit_transform(X_train_scaled[['age','bmi','children']])
    X_validate_scaled[['age','bmi','children']] = scaler.transform(X_validate_scaled[['age','bmi','children']])
    X_test_scaled[['age','bmi','children']] = scaler.transform(X_test_scaled[['age','bmi','children']])
    
    return X_train_scaled, X_validate, X_test

In [33]:
X_train_scaled, X_validate, X_test = scaled_data(X_train, X_validate, X_test)

In [34]:
X_train_scaled = X_train.copy()
X_validate_scaled = X_validate.copy()
X_test_scaled = X_test.copy()


scaler = StandardScaler()

X_train_scaled[['age','bmi','children']] = scaler.fit_transform(X_train_scaled[['age','bmi','children']])
X_validate_scaled[['age','bmi','children']] = scaler.transform(X_validate_scaled[['age','bmi','children']])
X_test_scaled[['age','bmi','children']] = scaler.transform(X_test_scaled[['age','bmi','children']])

X_train_scaled

,age,bmi,children,sex_encoded,smoker_encoded,northeast,northwest,southeast,southwest
463,1.182926,-0.802371,-0.878534,1,0,1,0,0,0
315,0.898367,0.421054,-0.878534,1,0,1,0,0,0
71,-0.595567,-0.373378,3.200375,1,0,1,0,0,0
741,-0.880126,-0.264666,-0.878534,1,1,0,0,1,0
423,-1.022406,-0.023828,-0.878534,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
481,0.684948,1.133534,0.753029,1,0,0,0,1,0
552,1.609765,-1.560845,-0.878534,1,0,0,0,0,1
1213,0.898367,0.429417,0.753029,0,0,0,0,0,1
418,1.752045,1.409495,-0.062752,1,0,0,0,1,0


In [35]:
def log_tran(y_train, y_validate):
    y_train_log = np.log1p(y_train)
    y_validate_log = np.log1p(y_validate)

    return y_train_log, y_validate_log

In [36]:
y_train_log = np.log1p(y_train)
y_validate_log = np.log1p(y_validate)


In [37]:
y_train_log.mean()

np.float64(9.064400266552193)

In [38]:
# Put logged y into DataFrames for convenience
base_y_train = pd.DataFrame({'charges_log': y_train_log})
base_y_validate = pd.DataFrame({'charges_log': y_validate_log})

# --- Baseline predictions in LOG space ---
pred_mean_log = y_train_log.mean()
pred_median_log = y_train_log.median()

base_y_train['pred_mean_log'] = pred_mean_log
base_y_validate['pred_mean_log'] = pred_mean_log

base_y_train['pred_median_log'] = pred_median_log
base_y_validate['pred_median_log'] = pred_median_log

# --- RMSE in LOG space ---
rmse_train_mean_log = mean_squared_error(base_y_train['charges_log'], base_y_train['pred_mean_log'])**(1/2)
rmse_validate_mean_log = mean_squared_error(base_y_validate['charges_log'], base_y_validate['pred_mean_log'])**(1/2)

rmse_train_median_log = mean_squared_error(base_y_train['charges_log'], base_y_train['pred_median_log'])**(1/2)
rmse_validate_median_log = mean_squared_error(base_y_validate['charges_log'], base_y_validate['pred_median_log'])**(1/2)

print("Baseline RMSE (log scale)")
print(f"Mean   - Train: {rmse_train_mean_log:.4f} | Validate: {rmse_validate_mean_log:.4f}")
print(f"Median - Train: {rmse_train_median_log:.4f} | Validate: {rmse_validate_median_log:.4f}")

# --- Convert baseline predictions back to DOLLARS ---
base_y_train['base_mean_dollars'] = np.expm1(base_y_train['pred_mean_log'])
base_y_validate['base_mean_dollars'] = np.expm1(base_y_validate['pred_mean_log'])

base_y_train['base_median_dollars'] = np.expm1(base_y_train['pred_median_log'])
base_y_validate['base_median_dollars'] = np.expm1(base_y_validate['pred_median_log'])

# --- RMSE in DOLLARS ---
rmse_train_mean_dollars = mean_squared_error(y_train, base_y_train.base_mean_dollars)**(1/2)
rmse_validate_mean_dollars = mean_squared_error(y_validate, base_y_validate.base_mean_dollars)**(1/2)

rmse_train_median_dollars = mean_squared_error(y_train, base_y_train.base_median_dollars)**(1/2)
rmse_validate_median_dollars = mean_squared_error(y_validate, base_y_validate.base_median_dollars)**(1/2)

print("\nBaseline RMSE (dollars)")
print(f"Mean   - Train: ${rmse_train_mean_dollars:,.2f} | Validate: ${rmse_validate_mean_dollars:,.2f}")
print(f"Median - Train: ${rmse_train_median_dollars:,.2f} | Validate: ${rmse_validate_median_dollars:,.2f}")



baseline_df = pd.DataFrame({
    'Train_Log': [rmse_train_mean_log, rmse_train_median_log],
    'Validate_Log': [rmse_validate_mean_log, rmse_validate_median_log],
    'Train_Dollars': [rmse_train_mean_dollars, rmse_train_median_dollars],
    'Validate_Dollars': [rmse_validate_mean_dollars, rmse_validate_median_dollars]
}, index=['Mean Baseline', 'Median Baseline'])

baseline_df

Baseline RMSE (log scale)
Mean   - Train: 0.8999 | Validate: 0.9788
Median - Train: 0.9024 | Validate: 0.9769

Baseline RMSE (dollars)
Mean   - Train: $12,198.50 | Validate: $14,343.50
Median - Train: $12,015.51 | Validate: $14,121.89


,Train_Log,Validate_Log,Train_Dollars,Validate_Dollars
Mean Baseline,0.899949,0.978814,12198.495794,14343.499589
Median Baseline,0.902419,0.976881,12015.511161,14121.892882


In [39]:
def baseline_rmse_log_and_dollars(
    y_train,
    y_validate,
    y_train_log,
    y_validate_log,
    log_transform: str = "log1p",
    return_preds: bool = False
):
    """
    Compute baseline RMSE in log space and in dollars for mean and median baselines.

    Parameters
    ----------
    y_train, y_validate : array-like
        Original target values in dollars.
    y_train_log, y_validate_log : array-like
        Log-transformed targets (must correspond to y_train/y_validate).
    log_transform : {"log1p", "log"}
        Which inverse transform to use when converting predictions back to dollars.
        - "log1p" assumes y_log = np.log1p(y) and uses np.expm1 to invert.
        - "log"   assumes y_log = np.log(y)   and uses np.exp to invert.
    return_preds : bool
        If True, returns the train/validate prediction DataFrames too.

    Returns
    -------
    baseline_df : pd.DataFrame
        RMSE summary table with columns:
        Train_Log, Validate_Log, Train_Dollars, Validate_Dollars
        and rows:
        Mean Baseline, Median Baseline
    (optional) base_y_train, base_y_validate : pd.DataFrame
        Only returned if return_preds=True
    """

    # Ensure series for consistent behavior
    y_train = pd.Series(y_train).astype(float)
    y_validate = pd.Series(y_validate).astype(float)
    y_train_log = pd.Series(y_train_log).astype(float)
    y_validate_log = pd.Series(y_validate_log).astype(float)

    # Choose inverse transform
    if log_transform == "log1p":
        inv = np.expm1
    elif log_transform == "log":
        inv = np.exp
    else:
        raise ValueError("log_transform must be 'log1p' or 'log'.")

    # Put logged y into DataFrames for convenience
    base_y_train = pd.DataFrame({'charges_log': y_train_log})
    base_y_validate = pd.DataFrame({'charges_log': y_validate_log})

    # --- Baseline predictions in LOG space ---
    pred_mean_log = y_train_log.mean()
    pred_median_log = y_train_log.median()

    base_y_train['pred_mean_log'] = pred_mean_log
    base_y_validate['pred_mean_log'] = pred_mean_log

    base_y_train['pred_median_log'] = pred_median_log
    base_y_validate['pred_median_log'] = pred_median_log

    # --- RMSE in LOG space ---
    rmse_train_mean_log = mean_squared_error(
        base_y_train['charges_log'], base_y_train['pred_mean_log'])**(1/2)
    rmse_validate_mean_log = mean_squared_error(
        base_y_validate['charges_log'], base_y_validate['pred_mean_log'])**(1/2)

    rmse_train_median_log = mean_squared_error(
        base_y_train['charges_log'], base_y_train['pred_median_log'])**(1/2)
    rmse_validate_median_log = mean_squared_error(
        base_y_validate['charges_log'], base_y_validate['pred_median_log'])**(1/2)

    # --- Convert baseline predictions back to DOLLARS ---
    base_y_train['base_mean_dollars'] = inv(base_y_train['pred_mean_log'])
    base_y_validate['base_mean_dollars'] = inv(base_y_validate['pred_mean_log'])

    base_y_train['base_median_dollars'] = inv(base_y_train['pred_median_log'])
    base_y_validate['base_median_dollars'] = inv(base_y_validate['pred_median_log'])

    # --- RMSE in DOLLARS ---
    rmse_train_mean_dollars = mean_squared_error(
        y_train, base_y_train['base_mean_dollars'])**(1/2)
    rmse_validate_mean_dollars = mean_squared_error(
        y_validate, base_y_validate['base_mean_dollars'])**(1/2)

    rmse_train_median_dollars = mean_squared_error(
        y_train, base_y_train['base_median_dollars'])**(1/2)
    rmse_validate_median_dollars = mean_squared_error(
        y_validate, base_y_validate['base_median_dollars'])**(1/2)

    # Summary table
    baseline_df = pd.DataFrame({
        'Train_Log': [rmse_train_mean_log, rmse_train_median_log],
        'Validate_Log': [rmse_validate_mean_log, rmse_validate_median_log],
        'Train_Dollars': [rmse_train_mean_dollars, rmse_train_median_dollars],
        'Validate_Dollars': [rmse_validate_mean_dollars, rmse_validate_median_dollars]
    }, index=['Mean Baseline', 'Median Baseline'])

    if return_preds:
        return baseline_df, base_y_train, base_y_validate
    return baseline_df

In [40]:
baseline_df = baseline_rmse_log_and_dollars(
    y_train, y_validate,
    y_train_log, y_validate_log,
    log_transform="log1p"
)
baseline_df


,Train_Log,Validate_Log,Train_Dollars,Validate_Dollars
Mean Baseline,0.899949,0.978814,12198.495794,14343.499589
Median Baseline,0.902419,0.976881,12015.511161,14121.892882


In [41]:
# Put logged y into DataFrames for convenience
linear_y_train = pd.DataFrame({'charges_log': y_train_log})
linear_y_validate = pd.DataFrame({'charges_log': y_validate_log})


# create the model object
lm = LinearRegression()

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm.fit(X_train_scaled, linear_y_train.charges_log)

# predict train
linear_y_train['pred_lm'] = lm.predict(X_train_scaled)

# evaluate: rmse
rmse_train = mean_squared_error(linear_y_train.charges_log, linear_y_train.pred_lm)**(1/2)

# predict validate
linear_y_validate['pred_lm'] = lm.predict(X_validate_scaled)

# evaluate: rmse
rmse_validate = mean_squared_error(linear_y_validate.charges_log, linear_y_validate.pred_lm)**(1/2)


# Convert predictions back to dollars
linear_y_train['pred_dollars'] = np.expm1(linear_y_train['pred_lm'])
linear_y_validate['pred_dollars'] = np.expm1(linear_y_validate['pred_lm'])

# evaluate: rmse in dollars
# train
rmse_train_dollars = mean_squared_error(y_train, linear_y_train['pred_dollars'])**(1/2)

# validate
rmse_validate_dollars = mean_squared_error(y_validate, linear_y_validate['pred_dollars'])**(1/2)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)
print("\n")
print("RMSE in Dollars")
print(f"Train: ${round(rmse_train_dollars, 2)}")
print(f"Validate: ${round(rmse_validate_dollars, 2)}")

linear_df = pd.DataFrame({
    'Train_Log': [rmse_train],
    'Validate_Log': [rmse_validate],
    'Train_Dollars': [rmse_train_dollars],
    'Validate_Dollars': [rmse_validate_dollars]
}, index=['linear_regression'])

linear_df

RMSE for OLS using LinearRegression
Training/In-Sample:  0.41272005156274627 
Validation/Out-of-Sample:  0.5125243131208743


RMSE in Dollars
Train: $8232.95
Validate: $9894.56


,Train_Log,Validate_Log,Train_Dollars,Validate_Dollars
linear_regression,0.41272,0.512524,8232.950363,9894.557731


In [42]:
# Put logged y into DataFrames for convenience
lars_y_train = pd.DataFrame({'charges_log': y_train_log})
lars_y_validate = pd.DataFrame({'charges_log': y_validate_log})

# create the model object
lars = LassoLars(alpha=1.0)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lars.fit(X_train_scaled, lars_y_train.charges_log)

# predict train
lars_y_train['pred_lars'] = lars.predict(X_train_scaled)

# evaluate: rmse
rmse_train = mean_squared_error(lars_y_train.charges_log, lars_y_train.pred_lars)**(1/2)

# predict validate
lars_y_validate['pred_lars'] = lars.predict(X_validate_scaled)

# evaluate: rmse
rmse_validate = mean_squared_error(lars_y_validate.charges_log, lars_y_validate.pred_lars)**(1/2)


# Convert predictions back to dollars
lars_y_train['pred_dollars'] = np.expm1(lars_y_train['pred_lars'])
lars_y_validate['pred_dollars'] = np.expm1(lars_y_validate['pred_lars'])

# evaluate: rmse in dollars
# train
rmse_train_dollars = mean_squared_error(y_train, lars_y_train['pred_dollars'])**(1/2)

# validate
rmse_validate_dollars = mean_squared_error(y_validate, lars_y_validate['pred_dollars'])**(1/2)


print("RMSE for Lasso + Lars\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)
print("\n")
print("RMSE in Dollars")
print(f"Train: ${round(rmse_train_dollars, 2)}")
print(f"Validate: ${round(rmse_validate_dollars, 2)}")


lasso_df = pd.DataFrame({
    'Train_Log': [rmse_train],
    'Validate_Log': [rmse_validate],
    'Train_Dollars': [rmse_train_dollars],
    'Validate_Dollars': [rmse_validate_dollars]
}, index=['lasso'])

lasso_df

RMSE for Lasso + Lars
Training/In-Sample:  0.8999488362007172 
Validation/Out-of-Sample:  0.9788138556303694


RMSE in Dollars
Train: $12198.5
Validate: $14343.5


,Train_Log,Validate_Log,Train_Dollars,Validate_Dollars
lasso,0.899949,0.978814,12198.495794,14343.499589


In [49]:
# Put logged y into DataFrames for convenience
tweedie_y_train = pd.DataFrame({'charges_log': y_train_log})
tweedie_y_validate = pd.DataFrame({'charges_log': y_validate_log})


# create the model object
glm = TweedieRegressor(power=1, alpha=0)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
glm.fit(X_train_scaled, tweedie_y_train.charges_log)

# predict train
tweedie_y_train['pred_glm'] = glm.predict(X_train_scaled)

# evaluate: rmse
rmse_train = mean_squared_error(tweedie_y_train.charges_log, tweedie_y_train.pred_glm)**(1/2)

# predict validate
tweedie_y_validate['pred_glm'] = glm.predict(X_validate_scaled)

# evaluate: rmse
rmse_validate = mean_squared_error(tweedie_y_validate.charges_log, tweedie_y_validate.pred_glm)**(1/2)

print("RMSE for GLM using Tweedie, power=1 & alpha=0\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)


# Convert predictions back to dollars
tweedie_y_train['pred_dollars'] = np.expm1(tweedie_y_train['pred_glm'])
tweedie_y_validate['pred_dollars'] = np.expm1(tweedie_y_validate['pred_glm'])

# evaluate: rmse in dollars
# train
rmse_train_dollars = mean_squared_error(y_train, tweedie_y_train['pred_dollars'])**(1/2)

# validate
rmse_validate_dollars = mean_squared_error(y_validate, tweedie_y_validate['pred_dollars'])**(1/2)




print("\n")
print("RMSE in Dollars")
print(f"Train: ${round(rmse_train_dollars, 2)}")
print(f"Validate: ${round(rmse_validate_dollars, 2)}")

tweedie_df = pd.DataFrame({
    'Train_Log': [rmse_train],
    'Validate_Log': [rmse_validate],
    'Train_Dollars': [rmse_train_dollars],
    'Validate_Dollars': [rmse_validate_dollars]
}, index=['tweedie'])

tweedie_df


RMSE for GLM using Tweedie, power=1 & alpha=0
Training/In-Sample:  0.4320819057951647 
Validation/Out-of-Sample:  0.5307844443994773


RMSE in Dollars
Train: $9600.62
Validate: $11620.53


,Train_Log,Validate_Log,Train_Dollars,Validate_Dollars
tweedie,0.432082,0.530784,9600.621331,11620.530347


#### Polynomial Regression 

In [44]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree=2)

# fit and transform X_train_scaled
X_train_degree2 = pf.fit_transform(X_train_scaled)

# transform X_validate_scaled & X_test_scaled
X_validate_degree2 = pf.transform(X_validate_scaled)
X_test_degree2 = pf.transform(X_test_scaled)

In [45]:
# Put logged y into DataFrames for convenience
lm2_y_train = pd.DataFrame({'charges_log': y_train_log})
lm2_y_validate = pd.DataFrame({'charges_log': y_validate_log})


# create the model object
lm2 = LinearRegression()

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm2.fit(X_train_degree2, lm2_y_train.charges_log)

# predict train
lm2_y_train['pred_lm2'] = lm2.predict(X_train_degree2)

# evaluate: rmse
rmse_train = mean_squared_error(lm2_y_train.charges_log, lm2_y_train.pred_lm2)**(1/2)

# predict validate
lm2_y_validate['pred_lm2'] = lm2.predict(X_validate_degree2)

# evaluate: rmse
rmse_validate = mean_squared_error(lm2_y_validate.charges_log, lm2_y_validate.pred_lm2)**(1/2)

print("RMSE for Polynomial Model, degrees=2\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)


# Convert predictions back to dollars
lm2_y_train['pred_dollars'] = np.expm1(lm2_y_train['pred_lm2'])
lm2_y_validate['pred_dollars'] = np.expm1(lm2_y_validate['pred_lm2'])

# evaluate: rmse in dollars
# train
rmse_train_dollars = mean_squared_error(y_train, lm2_y_train['pred_dollars'])**(1/2)

# validate
rmse_validate_dollars = mean_squared_error(y_validate, lm2_y_validate['pred_dollars'])**(1/2)




print("\n")
print("RMSE in Dollars")
print(f"Train: ${round(rmse_train_dollars, 2)}")
print(f"Validate: ${round(rmse_validate_dollars, 2)}")


poly_df = pd.DataFrame({
    'Train_Log': [rmse_train],
    'Validate_Log': [rmse_validate],
    'Train_Dollars': [rmse_train_dollars],
    'Validate_Dollars': [rmse_validate_dollars]
}, index=['polynomial'])

poly_df

RMSE for Polynomial Model, degrees=2
Training/In-Sample:  0.32720413410626215 
Validation/Out-of-Sample:  0.4349071848955806


RMSE in Dollars
Train: $4565.37
Validate: $6277.46


,Train_Log,Validate_Log,Train_Dollars,Validate_Dollars
polynomial,0.327204,0.434907,4565.373974,6277.455744


In [46]:
def run_regression_models(
    X_train_scaled, X_validate_scaled, X_test_scaled,
    y_train, y_validate, y_test,
    y_train_log, y_validate_log, y_test_log=None,
    baseline_df=None,
    poly_degree=2,
    tweedie_power=1,
    tweedie_alpha=0,
    lars_alpha=1.0,
    log_transform="log1p",
    sort_by="Validate_Dollars",
    return_models=False,
    return_preds=False
):
    """
    Fit multiple regression models on log-transformed target and evaluate RMSE
    in both log space and dollars.

    Models:
      - LinearRegression
      - LassoLars
      - TweedieRegressor
      - PolynomialFeatures(degree=poly_degree) + LinearRegression

    Parameters
    ----------
    X_*_scaled : array-like
        Scaled feature matrices (train/validate/test).
    y_* : array-like
        Original target in dollars.
    y_*_log : array-like
        Log-transformed targets corresponding to y_*.
    baseline_df : pd.DataFrame or None
        If you already computed baseline_df, pass it in to include in results.
    log_transform : {"log1p", "log"}
        Determines inverse transform for converting predictions back to dollars.
    sort_by : str
        Column to sort final results by.
    return_models : bool
        If True, returns a dict of fitted model objects.
    return_preds : bool
        If True, returns a dict with prediction DataFrames per model.

    Returns
    -------
    results_df : pd.DataFrame
    (optional) models : dict
    (optional) preds : dict
    """

    # Ensure series
    y_train = pd.Series(y_train).astype(float)
    y_validate = pd.Series(y_validate).astype(float)
    y_test = pd.Series(y_test).astype(float)

    y_train_log = pd.Series(y_train_log).astype(float)
    y_validate_log = pd.Series(y_validate_log).astype(float)
    if y_test_log is not None:
        y_test_log = pd.Series(y_test_log).astype(float)

    # Inverse transform
    if log_transform == "log1p":
        inv = np.expm1
    elif log_transform == "log":
        inv = np.exp
    else:
        raise ValueError("log_transform must be 'log1p' or 'log'.")

    def _rmse(y_true, y_pred):
        return mean_squared_error(y_true, y_pred)**(1/2)

    def _fit_predict_eval(model, name, Xtr, Xv, ytr_log, yv_log, ytr, yv):
        # fit
        model.fit(Xtr, ytr_log)

        # predict in log space
        pred_tr_log = model.predict(Xtr)
        pred_v_log = model.predict(Xv)

        # rmse in log space
        rmse_tr_log = _rmse(ytr_log, pred_tr_log)
        rmse_v_log  = _rmse(yv_log, pred_v_log)

        # convert to dollars
        pred_tr_dol = inv(pred_tr_log)
        pred_v_dol  = inv(pred_v_log)

        # rmse in dollars
        rmse_tr_dol = _rmse(ytr, pred_tr_dol)
        rmse_v_dol  = _rmse(yv, pred_v_dol)

        row = pd.DataFrame({
            "Train_Log": [rmse_tr_log],
            "Validate_Log": [rmse_v_log],
            "Train_Dollars": [rmse_tr_dol],
            "Validate_Dollars": [rmse_v_dol],
        }, index=[name])

        pred_df = {
            "train": pd.DataFrame({
                "y_log": ytr_log,
                "pred_log": pred_tr_log,
                "pred_dollars": pred_tr_dol
            }),
            "validate": pd.DataFrame({
                "y_log": yv_log,
                "pred_log": pred_v_log,
                "pred_dollars": pred_v_dol
            }),
        }

        return row, model, pred_df

    results = []
    models = {}
    preds = {}

    # Include baseline if provided
    if baseline_df is not None:
        results.append(baseline_df)

    # 1) Linear Regression
    lm = LinearRegression()
    row, fitted, pred_df = _fit_predict_eval(
        lm, "linear_regression",
        X_train_scaled, X_validate_scaled,
        y_train_log, y_validate_log,
        y_train, y_validate
    )
    results.append(row)
    models["linear_regression"] = fitted
    preds["linear_regression"] = pred_df

    # 2) LassoLars
    lars = LassoLars(alpha=lars_alpha)
    row, fitted, pred_df = _fit_predict_eval(
        lars, "lasso_lars",
        X_train_scaled, X_validate_scaled,
        y_train_log, y_validate_log,
        y_train, y_validate
    )
    results.append(row)
    models["lasso_lars"] = fitted
    preds["lasso_lars"] = pred_df

    # 3) Tweedie
    glm = TweedieRegressor(power=tweedie_power, alpha=tweedie_alpha)
    row, fitted, pred_df = _fit_predict_eval(
        glm, f"tweedie(p={tweedie_power},a={tweedie_alpha})",
        X_train_scaled, X_validate_scaled,
        y_train_log, y_validate_log,
        y_train, y_validate
    )
    results.append(row)
    models["tweedie"] = fitted
    preds["tweedie"] = pred_df

    # 4) Polynomial degree N + Linear Regression
    pf = PolynomialFeatures(degree=poly_degree, include_bias=False)
    Xtr_poly = pf.fit_transform(X_train_scaled)
    Xv_poly  = pf.transform(X_validate_scaled)
    Xt_poly  = pf.transform(X_test_scaled)  # ready if you want later

    lm_poly = LinearRegression()
    row, fitted, pred_df = _fit_predict_eval(
        lm_poly, f"polynomial_deg_{poly_degree}",
        Xtr_poly, Xv_poly,
        y_train_log, y_validate_log,
        y_train, y_validate
    )
    results.append(row)
    models[f"polynomial_deg_{poly_degree}"] = fitted
    models["poly_transform"] = pf
    preds[f"polynomial_deg_{poly_degree}"] = pred_df

    # Combine results
    results_df = pd.concat(results, axis=0)

    # Sort
    if sort_by in results_df.columns:
        results_df = results_df.sort_values(sort_by, ascending=True)
    else:
        results_df = results_df.sort_values("Validate_Dollars", ascending=True)

    if return_models and return_preds:
        return results_df, models, preds
    if return_models:
        return results_df, models
    if return_preds:
        return results_df, preds
    return results_df


In [47]:
models_df = run_regression_models(
    X_train_scaled, X_validate_scaled, X_test_scaled,
    y_train, y_validate, y_test,
    y_train_log, y_validate_log,
    baseline_df=baseline_df,
    poly_degree=2,
    log_transform="log1p",
    sort_by="Validate_Dollars"
)

models_df


,Train_Log,Validate_Log,Train_Dollars,Validate_Dollars
polynomial_deg_2,0.327204,0.434907,4565.373974,6277.455744
linear_regression,0.412720,0.512524,8232.950363,9894.557731
"tweedie(p=1,a=0)",0.432082,0.530784,9600.621331,11620.530347
Median Baseline,0.902419,0.976881,12015.511161,14121.892882
Mean Baseline,0.899949,0.978814,12198.495794,14343.499589
lasso_lars,0.899949,0.978814,12198.495794,14343.499589


In [50]:
models_df = pd.concat([baseline_df,linear_df,lasso_df,tweedie_df,poly_df], axis=0)

models_df.sort_values('Train_Dollars', ascending = True)

,Train_Log,Validate_Log,Train_Dollars,Validate_Dollars
polynomial,0.327204,0.434907,4565.373974,6277.455744
linear_regression,0.412720,0.512524,8232.950363,9894.557731
tweedie,0.432082,0.530784,9600.621331,11620.530347
Median Baseline,0.902419,0.976881,12015.511161,14121.892882
Mean Baseline,0.899949,0.978814,12198.495794,14343.499589
lasso,0.899949,0.978814,12198.495794,14343.499589


#### Takeaways:

- The polynomial regression model achieved the lowest validation RMSE in dollar terms, indicating the best predictive performance among the evaluated models.